In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import os
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, minmax_scale
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import pandas as pd

In [2]:
pages = []

for i in range(1, 102):
    pages.append(f"https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page={i}")
print(pages)

['https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=1', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=2', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=3', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=4', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=5', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=6', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=7', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=8', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=9', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=10', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=11', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=12', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=13', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=14', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=15', 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom

In [ ]:
# Tworzenie folderu na obrazy
if not os.path.exists("images"):
    os.makedirs("images")

In [ ]:
# Funkcja do pobierania szczegółowych danych zwierzęcia (wraz z opisem)
def scrape_pet_details(pet_url):
    response = requests.get(pet_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            # Pobranie sekcji z nazwą zwierzaka
            name_div = soup.find('div', {'class': 'name'})
            name = name_div.h3.text.strip() if name_div and name_div.h3 else "Brak nazwy"

            # Pobranie opisu zwierzęcia
            description_div = soup.find('div', {'class': 'pet-description markdownit'})
            opis = description_div.text.strip() if description_div else "Brak opisu"

            # Pobranie pozostałych szczegółów
            details_ul = soup.find('ul', {'class': 'petdetails'})
            if details_ul:
                details = details_ul.find_all('li')
                pet_info = {}
                for detail in details:
                    text = detail.text.strip()
                    if "W typie rasy:" in text:
                        pet_info['rasa'] = text.replace("W typie rasy:", "").strip()
                    elif "Wiek:" in text:
                        pet_info['wiek'] = text.replace("Wiek:", "").strip()
                    elif "Płeć:" in text:
                        pet_info['płeć'] = text.replace("Płeć:", "").strip()
                    elif "Waga:" in text:
                        pet_info['waga'] = text.replace("Waga:", "").strip()
                    elif "Nr:" in text:
                        pet_info['numer'] = text.replace("Nr:", "").strip()
                    elif "Status:" in text:
                        pet_info['status'] = text.replace("Status:", "").strip()
                    elif "Przyjęty:" in text:
                        pet_info['przyjęty'] = text.replace("Przyjęty:", "").strip()
                    elif "Wydany:" in text:
                        pet_info['wydany'] = text.replace("Wydany:", "").strip()
                    elif "Znaleziony:" in text:
                        pet_info['znaleziony'] = text.replace("Znaleziony:", "").strip()

                # Łączenie danych zwierzęcia w jeden słownik
                pet_data = {
                    'name': name,
                    'opis': opis,
                    **pet_info
                }

                return pet_data

            else:
                print(f"Brak szczegółowych danych w {pet_url}")
                return {'name': name, 'opis': opis}

        except Exception as e:
            print(f"Błąd podczas pobierania danych szczegółowych: {e}")
            return None

    else:
        print(f"Nie udało się pobrać szczegółów zwierzęcia: {pet_url}")
        return None

In [ ]:
global_file_counter = 1 # Wartość początkowa do numerowania zdjęć
all_pets = []

In [ ]:
for pag in pages:
    print(f"Pobieram stronę: {pag}")
    page = requests.get(pag)
    if page.status_code == 200:
        soup = BeautifulSoup(page.text, 'html.parser')

        # Pobieranie obrazów
        image_divs = soup.find_all('div', {'class': 'image'})
        for i, div in enumerate(image_divs):
            style = div.get('style')  # Jeśli obraz w tle jest w stylu CSS
            img_url = None

            if style and 'url' in style:
                img_url = style.split('url(')[1].split(')')[0].strip('"')
            else:
                img_tag = div.find('img')
                if img_tag:
                    img_url = img_tag.get('src')

            if img_url:
                if not img_url.startswith("http"):
                    img_url = f"https://napaluchu.waw.pl{img_url}"
                img_data = requests.get(img_url).content
                file_name = f"images/image_{global_file_counter}.jpg"
                global_file_counter += 1
                with open(file_name, 'wb') as f:
                    f.write(img_data)

        # Pobieranie linków do szczegółowych stron zwierząt
        pet_links = []
        pet_divs = soup.find_all('div', {'class': 'pet-block col-md-4 col-sm-12 col-xs-12'})
        for div in pet_divs:
            pet_link = div.find('a', href=True)
            if pet_link:
                full_link = f"https://napaluchu.waw.pl{pet_link['href']}"
                pet_links.append(full_link)

        print(f"Znaleziono {len(pet_links)} zwierząt na stronie.")

        # Pobieranie szczegółowych danych o zwierzętach
        for pet_url in pet_links:
            pet_details = scrape_pet_details(pet_url)
            if pet_details:
                all_pets.append(pet_details)

            # Krótka przerwa, aby uniknąć przeciążenia serwera
            time.sleep(5)

    else:
        print(f"Nie udało się pobrać strony: {pag}")

    # Przerwa między stronami
    time.sleep(60)

# Tworzenie DataFrame z listy słowników
df = pd.DataFrame(all_pets)

# Zapis do pliku CSV
df.to_csv("pets_data.csv", index=False, encoding='utf-8')

print("Pobieranie zakończone. Dane zapisane w 'pets_data.csv'.")

Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=202
Znaleziono 15 zwierząt na stronie.
Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=203
Znaleziono 15 zwierząt na stronie.
Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=204
Znaleziono 15 zwierząt na stronie.
Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=205
Znaleziono 15 zwierząt na stronie.
Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=206
Znaleziono 15 zwierząt na stronie.
Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=207
Znaleziono 15 zwierząt na stronie.
Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=208
Znaleziono 15 zwierząt na stronie.
Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=209
Znaleziono 15 zwierząt na stronie.
Pobieram stronę: https://napaluchu.waw.pl/zwierzeta/znalazly-dom/?pet_page=210
Znalezion

In [ ]:
# FOLDER W ZIP, ABY MOŻNA BYŁO POBRAĆ
import shutil

folder_to_zip = "images"
output_zip = "images.zip"

shutil.make_archive(output_zip.replace(".zip", ""), 'zip', folder_to_zip)

'/content/images.zip'